In [48]:
import gc
import re
import nltk
import faiss
import importlib

import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from collections import defaultdict, Counter

import warnings
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/ma/ma_ma/ma_pbhattar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
df_sess = pd.read_csv("../../data/train/sessions_train.csv")
df_sess

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
3606244,['B086CYFSKW' 'B0874F9859' 'B086CYFSKW'],B07B5TYD76,IT
3606245,['B09NRZKZ7V' 'B08WJTPV93'],B08L1P4C3D,IT
3606246,['B085JFX7MP' 'B085JGHW8R'],B01MPWVD44,IT
3606247,['B00B0UING2' 'B00B0UING2'],B00D3HYEZ4,IT


In [50]:
df_test = pd.read_csv("../../data/test/sessions_test_task1_phase2.csv")
df_test

,prev_items,locale
0,['B087VLP2RT' 'B09BRQSHYH' 'B099KW4ZLV'],DE
1,['B08XW4W667' 'B096VMCJYF' 'B096VMCJYF'],DE
2,['B09Z4T2GJ3' 'B09Z3FBXMB' 'B0936K9LTJ' 'B09Z4...,DE
3,['B07T6Y2HG7' 'B07T2NBLX9' 'B07Y1G5F3Y'],DE
4,['B0B2DRKZ6X' 'B0B2DRKZ6X' 'B0B2DRKZ6X'],DE
...,...,...
316967,['B078RJX3CC' 'B07GKM97YF'],UK
316968,['B01LX5Y7RG' 'B00M35Y2J0' 'B0BFR9D1Y2' 'B09BB...,UK
316969,['B09HGRXXTM' 'B08VDNCZT9'],UK
316970,['B089CVQ2FS' 'B089CVQ2FS'],UK


In [51]:
# sessions_test = pd.read_csv("../../data/test/sessions_test_task1_phase2.csv")
# sessions_train = pd.read_csv("../../data/train/sessions_train.csv")
# products_train = pd.read_csv("../../data/train/products_train_processed.csv")

In [52]:
def str2list(x):
    x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
    l = [i for i in x.split() if i]
    return l

In [53]:
import data_utils
importlib.reload(data_utils)
from data_utils import split_locales

task1_locales = ['DE', 'JP', 'UK']
output_path = '../../outputs/'
st_de, st_jp, st_uk = split_locales(df_sess, task1_locales)

In [54]:
# def str2list(x, locale=None):
#     x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
#     l = [i for i in x.split() if i and i in st_de if locale=='DE' and i in st_jp if locale=='JP' and i in st_uk if locale=='UK']
#     return l

In [55]:
def run_model(df, df_test_locale):
    next_item_dict = defaultdict(list)

    for _, row in tqdm(df.iterrows(), total=len(df)):
        prev_items = str2list(row['prev_items'])
        next_item = row['next_item']
        prev_items_length = len(prev_items)
        if prev_items_length <= 1:
            next_item_dict[prev_items[0]].append(next_item)
        else:
            for i, item in enumerate(prev_items[:-1]):
                next_item_dict[item].append(prev_items[i+1])
            next_item_dict[prev_items[-1]].append(next_item)

    for _, row in tqdm(df_test_locale.iterrows(), total=len(df_test_locale)):
        prev_items = str2list(row['prev_items'])
        prev_items_length = len(prev_items)
        if prev_items_length <= 1:
            continue
        else:
            for i, item in enumerate(prev_items[:-1]):
                next_item_dict[item].append(prev_items[i+1])

    next_item_map = {}

    for item in tqdm(next_item_dict):
        counter = Counter(next_item_dict[item])
        next_item_map[item] = [i[0] for i in counter.most_common(100)]

    k = []
    v = []

    for item in next_item_dict:
        k.append(item)
        v.append(next_item_dict[item])

    df_next = pd.DataFrame({'item': k, 'next_item': v})
    df_next = df_next.explode('next_item').reset_index(drop=True)

    top200 = df_next['next_item'].value_counts().index.tolist()[:200]

    df_test_locale['last_item'] = df_test_locale['prev_items'].apply(lambda x: str2list(x)[-1])
    df_test_locale['next_item_prediction'] = df_test_locale['last_item'].map(next_item_map)

    preds = []

    for _, row in tqdm(df_test_locale.iterrows(), total=len(df_test_locale)):
        pred_orig = row['next_item_prediction']
        pred = pred_orig
        prev_items = str2list(row['prev_items'])
        if type(pred) == float:
            pred = top200[:100]
        else:
            if len(pred_orig) < 100:
                for i in top200:
                    if i not in pred_orig and i not in prev_items:
                        pred.append(i)
                    if len(pred) >= 100:
                        break
            else:
                pred = pred[:100]

        pred = [i for i in pred if i not in prev_items] # Filter out items that are already in prev_items

        # If after filtering pred has less than 100 items, fill up with items from top200
        if len(pred) < 100:
            for i in top200:
                if i not in pred and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        preds.append(pred)

    df_test_locale['next_item_prediction'] = preds

    return df_test_locale

In [56]:
# Create locale-specific test dataframes
df_test_de = df_test[df_test['locale'] == 'DE']
df_test_jp = df_test[df_test['locale'] == 'JP']
df_test_uk = df_test[df_test['locale'] == 'UK']

In [57]:
# Run the model for each locale
df_test_de = run_model(st_de, df_test_de)
df_test_jp = run_model(st_jp, df_test_jp)
df_test_uk = run_model(st_uk, df_test_uk)

  0%|          | 0/1111416 [00:00<?, ?it/s]

  0%|          | 0/104568 [00:00<?, ?it/s]

  0%|          | 0/488848 [00:00<?, ?it/s]

  0%|          | 0/104568 [00:00<?, ?it/s]

In [59]:
# Concatenate the results
df_test = pd.concat([df_test_de, df_test_jp, df_test_uk])
df_test

,prev_items,locale,last_item,next_item_prediction
0,['B087VLP2RT' 'B09BRQSHYH' 'B099KW4ZLV'],DE,B099KW4ZLV,"[B07SDFLVKD, B091CK241X, B0BGC82WVW, B0B9GJLV2..."
1,['B08XW4W667' 'B096VMCJYF' 'B096VMCJYF'],DE,B096VMCJYF,"[B084CB7GX9, B09YD8XV6M, B004P4QFJM, B004P4OF1..."
2,['B09Z4T2GJ3' 'B09Z3FBXMB' 'B0936K9LTJ' 'B09Z4...,DE,B09Z4PYG8Q,"[B09Z4PZQBF, B08LLF9M11, B01GS8K962, B08KHJN9H..."
3,['B07T6Y2HG7' 'B07T2NBLX9' 'B07Y1G5F3Y'],DE,B07Y1G5F3Y,"[B09GKJ9RRJ, B07X8MW1G1, B07QQZD49D, B07T5XJW9..."
4,['B0B2DRKZ6X' 'B0B2DRKZ6X' 'B0B2DRKZ6X'],DE,B0B2DRKZ6X,"[B0B2JY9THB, B08YK8FQJ8, B08SHZHRQ7, B09C89S7W..."
...,...,...,...,...
316967,['B078RJX3CC' 'B07GKM97YF'],UK,B07GKM97YF,"[B07GKP2LCF, B07GKYSHB4, B00V6FIFZ0, B006DDGCI..."
316968,['B01LX5Y7RG' 'B00M35Y2J0' 'B0BFR9D1Y2' 'B09BB...,UK,B000FHC0QK,"[B00M35Y326, B08B395NHL, B091DWY6CH, B085C7TCT..."
316969,['B09HGRXXTM' 'B08VDNCZT9'],UK,B08VDNCZT9,"[B08VDHH6QF, B08VDSL596, B08VD5DC5L, B07QK2SPP..."
316970,['B089CVQ2FS' 'B089CVQ2FS'],UK,B089CVQ2FS,"[B089CZWB4C, B08W2JJZBM, B08T1ZJYHV, B09WCQYGX..."


In [60]:
df_test['next_item_prediction'].apply(len).describe()

count    316972.0
mean        100.0
std           0.0
min         100.0
25%         100.0
50%         100.0
75%         100.0
max         100.0
Name: next_item_prediction, dtype: float64

In [61]:
df_test[['locale', 'next_item_prediction']].to_parquet('../../outputs/submission_task1_locale_new_updated.parquet', engine='pyarrow', compression='gzip')